In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import HTML, display
from sklearn.ensemble import BaggingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import datto as dt

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/kristiewirth/.pyenv/versions/mlrecommendations/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/kristiewirth/.pyenv/versions/mlrecommendations/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


## Data Connections

In [2]:
dc = dt.DataConnections()

In [3]:
conn = dc.setup_redshift_connection()

In [4]:
df = dc.run_sql_redshift(conn, """SELECT
    customuser_id,
    developer,
    has_sso_at_signup,
    days_between_signup_and_activation,
    role_name,
    org_name,
    followed_apps_count
FROM dim_user_all
WHERE
    (activation_date >= (TIMESTAMP '2020-01-01') AND (activation_date ) < (TIMESTAMP '2020-01-31'))
    AND (NOT COALESCE(is_spam, FALSE)) 
    AND (NOT COALESCE(is_employee, FALSE))
    AND current_version""")

In [5]:
df

,customuser_id,developer,has_sso_at_signup,days_between_signup_and_activation,role_name,org_name,followed_apps_count
0,10768,False,False,2722,None,None,11.0
1,37548,False,False,2569,None,None,12.0
2,42565,False,False,2539,None,None,17.0
3,42854,True,False,2552,None,None,8.0
4,55249,False,False,2520,None,None,14.0
...,...,...,...,...,...,...,...
41775,6602264,False,True,-1,None,None,10.0
41776,6602646,False,False,-1,None,None,2.0
41777,6603480,False,True,-1,Real Estate Agent,LEGF Constructora,9.0
41778,6603857,False,False,-1,Other,indonesia,1.0


**Can also save & load from S3!**

save_to_s3(directory_path, object_to_save, object_name)
    
load_from_s3(directory_path, object_name)

## Setup

In [6]:
s = dt.Setup()

In [7]:
s.setup_logger()

<RootLogger root (DEBUG)>

In [8]:
df

,customuser_id,developer,has_sso_at_signup,days_between_signup_and_activation,role_name,org_name,followed_apps_count
0,10768,False,False,2722,None,None,11.0
1,37548,False,False,2569,None,None,12.0
2,42565,False,False,2539,None,None,17.0
3,42854,True,False,2552,None,None,8.0
4,55249,False,False,2520,None,None,14.0
...,...,...,...,...,...,...,...
41775,6602264,False,True,-1,None,None,10.0
41776,6602646,False,False,-1,None,None,2.0
41777,6603480,False,True,-1,Real Estate Agent,LEGF Constructora,9.0
41778,6603857,False,False,-1,Other,indonesia,1.0


In [9]:
s.display_more_data(20)

## Front End

In [10]:
fe = dt.FrontEnd()

In [11]:
dropdown_html = fe.dropdown_from_dataframe('Role Name', df, 'role_name')

In [12]:
display(HTML(dropdown_html))

## Exploratory Data Analysis (EDA)

In [13]:
eda = dt.Eda()
numerical_vals, categorical_vals = eda.separate_cols_by_type(df)

In [14]:
numerical_vals

,customuser_id,days_between_signup_and_activation,followed_apps_count
0,10768,2722,11.000000
1,37548,2569,12.000000
2,42565,2539,17.000000
3,42854,2552,8.000000
4,55249,2520,14.000000
...,...,...,...
41775,6602264,-1,10.000000
41776,6602646,-1,2.000000
41777,6603480,-1,9.000000
41778,6603857,-1,1.000000


In [15]:
categorical_vals

,developer,has_sso_at_signup,role_name,org_name
0,False,False,None,None
1,False,False,None,None
2,False,False,None,None
3,True,False,None,None
4,False,False,None,None
...,...,...,...,...
41775,False,True,None,None
41776,False,False,None,None
41777,False,True,Real Estate Agent,LEGF Constructora
41778,False,False,Other,indonesia


In [16]:
eda.check_for_mistyped_booleans(numerical_vals)

[]

In [17]:
eda.find_cols_to_exclude(df)

customuser_id
role_name
org_name


['customuser_id', 'role_name', 'org_name']

In [18]:
eda.sample_unique_vals(df)

customuser_id
[ 10768  37548  42565  42854  55249  59641  63194  77771  88974  97967
 136683 146705 153523 156032 168040 182417 187831 222237 232958 236589]
41780

------------------------------------

developer
[False  True]
2

------------------------------------

has_sso_at_signup
[False  True]
2

------------------------------------

days_between_signup_and_activation
[2722 2569 2539 2552 2520 2492 2471 2425 2398 2348 2231 2209 2194 2200
 2188 2154 2159 2119 2083 2088]
1600

------------------------------------

role_name
[None 'DevOps Engineer' 'Business Owner or Founder' 'Other' 'Salesperson'
 'Information Technology Professional' 'Web Engineer'
 'Business Development Professional' 'Marketer' 'Designer'
 'President or CEO' 'Software Engineer' 'Operations Professional'
 'Marketing' 'Student' 'Teacher or Professor' 'Legal Professional'
 'Real Estate Agent' 'Consulting' 'Project Manager']
43

------------------------------------

org_name
[None 'MAS Digital Media' 'Canadian Baptist 

In [19]:
eda.find_correlated_features(df)

days_between_signup_and_activation  customuser_id                        0.963381
customuser_id                       days_between_signup_and_activation   0.963381
has_sso_at_signup                   followed_apps_count                  0.391084
followed_apps_count                 has_sso_at_signup                    0.391084
days_between_signup_and_activation  has_sso_at_signup                    0.159301
has_sso_at_signup                   days_between_signup_and_activation   0.159301
customuser_id                       has_sso_at_signup                    0.130802
has_sso_at_signup                   customuser_id                        0.130802
followed_apps_count                 developer                            0.071712
developer                           followed_apps_count                  0.071712
followed_apps_count                 customuser_id                        0.071237
customuser_id                       followed_apps_count                  0.071237
followed_apps_co

days_between_signup_and_activation  customuser_id                        0.963381
customuser_id                       days_between_signup_and_activation   0.963381
has_sso_at_signup                   followed_apps_count                  0.391084
followed_apps_count                 has_sso_at_signup                    0.391084
days_between_signup_and_activation  has_sso_at_signup                    0.159301
has_sso_at_signup                   days_between_signup_and_activation   0.159301
customuser_id                       has_sso_at_signup                    0.130802
has_sso_at_signup                   customuser_id                        0.130802
followed_apps_count                 developer                            0.071712
developer                           followed_apps_count                  0.071712
followed_apps_count                 customuser_id                        0.071237
customuser_id                       followed_apps_count                  0.071237
followed_apps_co

In [20]:
df["days_between_signup_and_activation"].fillna(
    df["days_between_signup_and_activation"].mean(), inplace=True
)
df["followed_apps_count"].fillna(df["followed_apps_count"].mean(), inplace=True)

df["role_name"].fillna("Unknown", inplace=True)
df["org_name"].fillna("Unknown", inplace=True)

## Clean Text

In [21]:
ct = dt.CleanText()

In [22]:
example1 = """I am some text with a link in it: https://www.google.com Sincerely Kristie"""

In [23]:
ct.remove_links(example1)

'I am some text with a link in it:  Sincerely Kristie'

In [24]:
example2 = """I enjoy reading and meditating. Yesterday I made a quiche."""

In [25]:
ct.lematize(example2)

['-PRON-',
 'enjoy',
 'read',
 'and',
 'meditating',
 '.',
 'yesterday',
 '-PRON-',
 'make',
 'a',
 'quiche',
 '.']

In [26]:
example3 = """
Dear so & so, 

I have a question for you. 

Sincerely, Kristie"""

In [27]:
ct.remove_email_greetings_signatures(example3)

'\n\n\nI have a question for you. \n\n'

In [28]:
ct.clean_column_names(df)

,customuser_id,developer,has_sso_at_signup,days_between_signup_and_activation,role_name,org_name,followed_apps_count
0,10768,False,False,2722,Unknown,Unknown,11.000000
1,37548,False,False,2569,Unknown,Unknown,12.000000
2,42565,False,False,2539,Unknown,Unknown,17.000000
3,42854,True,False,2552,Unknown,Unknown,8.000000
4,55249,False,False,2520,Unknown,Unknown,14.000000
...,...,...,...,...,...,...,...
41775,6602264,False,True,-1,Unknown,Unknown,10.000000
41776,6602646,False,False,-1,Unknown,Unknown,2.000000
41777,6603480,False,True,-1,Real Estate Agent,LEGF Constructora,9.000000
41778,6603857,False,False,-1,Other,indonesia,1.000000


In [29]:
df.columns = [x.replace('_', ' ').capitalize() for x in df.columns]
df.head(1)

,Customuser id,Developer,Has sso at signup,Days between signup and activation,Role name,Org name,Followed apps count
0,10768,False,False,2722,Unknown,Unknown,11.000000


In [30]:
df = ct.clean_column_names(df)
df.head(1)

,customuser_id,developer,has_sso_at_signup,days_between_signup_and_activation,role_name,org_name,followed_apps_count
0,10768,False,False,2722,Unknown,Unknown,11.000000


In [31]:
# Can also do 'datetime' and 'str'
ct.fix_col_data_type(df, 'followed_apps_count', 'int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41780 entries, 0 to 41779
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   customuser_id                       41780 non-null  int64  
 1   developer                           41780 non-null  bool   
 2   has_sso_at_signup                   41780 non-null  bool   
 3   days_between_signup_and_activation  41780 non-null  int64  
 4   role_name                           41780 non-null  object 
 5   org_name                            41780 non-null  object 
 6   followed_apps_count                 41780 non-null  float64
dtypes: bool(2), float64(1), int64(2), object(2)
memory usage: 1.7+ MB


In [32]:
dup_df = pd.DataFrame([[124, 124], [5676, 5676], [456, 456]], columns=['id', 'id'])
dup_df

,id,id
0,124,124
1,5676,5676
2,456,456


In [33]:
ct.remove_duplicate_columns(dup_df)

,id
0,124
1,5676
2,456


## Train Model

In [34]:
df = df.sample(5000)

In [35]:
tm = dt.TrainModel()

In [36]:
y = df['developer']
X = df.drop('developer', axis=1)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train.drop("customuser_id", inplace=True, axis=1)
X_test.drop("customuser_id", inplace=True, axis=1)

/Users/kristiewirth/.pyenv/versions/mlrecommendations/lib/python3.7/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
vectorizer = TfidfVectorizer(
    tokenizer=ct.lematize,
    # Means each word must appear in at least __ different documents
    min_df=2,
)

# Fit + transform = learn from train data, then create transformations
train_vectors = vectorizer.fit_transform(X_train["org_name"]).todense()

# Transform only - you don't want to learn anything from the test data
test_vectors = vectorizer.transform(X_test["org_name"]).todense()

words = vectorizer.get_feature_names()
X_train_vectors = pd.DataFrame(train_vectors, columns=words, index=X_train.index)
X_test_vectors = pd.DataFrame(test_vectors, columns=words, index=X_test.index)

X_train.drop("org_name", inplace=True, axis=1)
X_test.drop("org_name", inplace=True, axis=1)

In [39]:
X_train_dummies = pd.get_dummies(X_train, drop_first=False)
X_test_dummies = pd.get_dummies(X_test, drop_first=False).reindex(
    columns=X_train_dummies.columns, fill_value=0
)

In [40]:
X_train_combined = pd.concat([X_train_vectors, X_train_dummies], axis=1)
X_test_combined = pd.concat([X_test_vectors, X_test_dummies], axis=1)

In [41]:
pipeline = Pipeline([("model", LogisticRegression()),])

tm.model_testing(X_train_combined, y_train, pipeline, "classification", "precision")

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-2)]: Done 167 out of 180 | elapsed:   28.9s remaining:    2.2s
[Parallel(n_jobs=-2)]: Done 180 out of 180 | elapsed:   29.4s finished





*** Best Parameters Using ['precision', 'recall', 'roc_auc'] | Tie Breaker: precision | 2020-08-24 11:08 ***
Params: {'model': BaggingClassifier(), 'model__max_features': 1.0, 'model__n_estimators': 5}
Mean Recall: 0.0583
Mean Precision: 0.1011
Mean ROC AUC: 0.5082


Params: {'model': BaggingClassifier(), 'model__max_features': 1.0, 'model__n_estimators': 10}
Mean Recall: 0.0417
Mean Precision: 0.0923
Mean ROC AUC: 0.5281


Params: {'model': BaggingClassifier(), 'model__max_features': 1.0, 'model__n_estimators': 15}
Mean Recall: 0.0333
Mean Precision: 0.0702
Mean ROC AUC: 0.5598


Params: {'model': MLPClassifier(activation='identity', alpha=0.01), 'model__activation': 'identity', 'model__alpha': 0.01}
Mean Recall: 0.0169
Mean Precision: 0.1762
Mean ROC AUC: 0.5735


Params: {'model': MLPClassifier(activation='identity', alpha=0.01), 'model__activation': 'identity', 'model__alpha': 0.001}
Mean Recall: 0.0083
Mean Precision: 0.1667
Mean ROC AUC: 0.5550




{'model': MLPClassifier(activation='identity', alpha=0.01),
 'model__activation': 'identity',
 'model__alpha': 0.01}

## Model Results

### Classifier

In [42]:
pipeline = Pipeline([("model", BaggingClassifier(max_features=1.0, n_estimators=15))])

pipeline.fit(X_train_combined, y_train)
y_predicted = pipeline.predict(X_test_combined)

mr = dt.ModelResults()
mr.score_final_model(
    "classification", X_train_combined, y_train, X_test_combined, y_test, pipeline
)

Final Model Precision: 0.2
Final Model Recall: 0.12121212121212122
Final Model ROC AUC: 0.5540325191105798
Predicted  False  True 
Actual                 
False       1201     16
True          29      4
Predicted    False    True 
Actual                     
False     0.961000 0.013000
True      0.023000 0.003000


(Pipeline(steps=[('model', BaggingClassifier(n_estimators=15))]),
 array([False, False, False, ..., False, False, False]))

In [43]:
mr.coefficients_summary(X_train_combined, y_train, 3, 10, 'classification')

/Users/kristiewirth/.pyenv/versions/mlrecommendations/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/kristiewirth/.pyenv/versions/mlrecommendations/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver optio

Coefficients summary (descending by mean abs value):
                                     features      mean      std    median
65                            role_name_Other -1.264426 0.069507 -1.226036
57                      role_name_IT & DevOps  1.118502 0.032696  1.112836
67                 role_name_President or CEO -0.996857 0.217683 -0.902935
78                role_name_Software Engineer  0.960563 0.451627  1.060777
82                     role_name_Web Engineer  0.789950 0.063833  0.795755
44  role_name_Customer Service Representative -0.676443 0.020511 -0.665603
76                      role_name_Salesperson -0.668993 0.016706 -0.660566
46                         role_name_Designer -0.665688 0.025757 -0.656470
64          role_name_Operations Professional -0.651351 0.040500 -0.666726
60                         role_name_Marketer -0.575439 0.222738 -0.566887


,features,mean,std,median
65,role_name_Other,-1.264426,0.069507,-1.226036
57,role_name_IT & DevOps,1.118502,0.032696,1.112836
67,role_name_President or CEO,-0.996857,0.217683,-0.902935
78,role_name_Software Engineer,0.960563,0.451627,1.060777
82,role_name_Web Engineer,0.789950,0.063833,0.795755
44,role_name_Customer Service Representative,-0.676443,0.020511,-0.665603
76,role_name_Salesperson,-0.668993,0.016706,-0.660566
46,role_name_Designer,-0.665688,0.025757,-0.656470
64,role_name_Operations Professional,-0.651351,0.040500,-0.666726
60,role_name_Marketer,-0.575439,0.222738,-0.566887


### Text analysis

In [44]:
text_df = dc.run_sql_redshift(conn, """
SELECT
    body
FROM hs_threads
WHERE created_at > '2020-08-01' AND created_by_customer = True
LIMIT 100
""")

In [45]:
topic_words_df, combined_df = mr.most_similar_texts(text_df, 10, 10, 10, 'body')

/Users/kristiewirth/.pyenv/versions/mlrecommendations/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-PRON-', 'make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Topics created with top words & example texts:
   topic_num                                          top_words  \
0          0  [-PRON-, ., ,, !, thank, thank -PRON-, -PRON- ...   
1          1  [chris.allen@harvardbusiness.org, -, 617, 617 ...   
2          2  [|, -, mathias, :, password, mathias j., 650, ...   
3          3  [61 broadway suite, 448 - 3296, ) 448, 646 ) 4...   
4          4  [update ? thank, update ?, , naveen, naveen ko...   
5          5  [|, :, intend, : +91, +91, #, information, int...   
6          6  [=, = =, http://www.quickbuyproperties.co.za/,...   
7          7  [-PRON-, -PRON- credit, credit, , -PRON-, arvi...   
8          8  [thank update !, thank update, update !, updat...   
9          9            [", -PRON-, ,, :, ), ., >, ) :, -, zap]   

                                        sample_texts  
0  [Hi,\nI see that we now have 11 templates for ...  
1  [Hi - I want to get back to this. Is there ava...  
2  [Were you successful with resetting the passwo.

In [46]:
topic_words_df

,topic_num,top_words,sample_texts
0,0,"[-PRON-, ., ,, !, thank, thank -PRON-, -PRON- ...","[Hi,\nI see that we now have 11 templates for ..."
1,1,"[chris.allen@harvardbusiness.org, -, 617, 617 ...",[Hi - I want to get back to this. Is there ava...
2,2,"[|, -, mathias, :, password, mathias j., 650, ...",[Were you successful with resetting the passwo...
3,3,"[61 broadway suite, 448 - 3296, ) 448, 646 ) 4...",[Hello!\nI wanted to follow up on this to see ...
4,4,"[update ? thank, update ?, , naveen, naveen ko...","[Hi Andy,\nI was wondering what we needed to d..."
5,5,"[|, :, intend, : +91, +91, #, information, int...","[Hi Owari,\nYes we need to do the below operat..."
6,6,"[=, = =, http://www.quickbuyproperties.co.za/,...",[Dear Laura\nSee the screenshots\nThis from FB...
7,7,"[-PRON-, -PRON- credit, credit, , -PRON-, arvi...","[Hello Arvind,\nWe don't see the credit in the..."
8,8,"[thank update !, thank update, update !, updat...","[Hi Wes,\nI just checked in with Bullhorn for ..."
9,9,"["", -PRON-, ,, :, ), ., >, ) :, -, zap]","[Dear Kaushi,\nHope you are doing good.\nI've ..."


In [47]:
combined_df[['body', "top_topic_num"]]

,body,top_topic_num
0,"Hi,\nI see that we now have 11 templates for P...",0
1,"Hi Andy,\nI was wondering what we needed to do...",4
2,Hi Nick!\nIt´s me again! Hope you are very goo...,0
3,"Hello,\n \nI hope you and your team are safe a...",0
4,Hi\nWhen can I get the discount?\n‫בתאריך יום ...,0
...,...,...
95,"Lara,\nNowhere in zoho does it have anything a...",0
96,yep we already sent a screenshot here it is ag...,0
97,ok just reconnected.\nnot to add more to your ...,0
98,its lead status not lead source,0


In [48]:
mr.most_common_words_by_group(combined_df, 'body', 'top_topic_num', 10, 2, 2)

Group Name: 0

pellcomp uk 6
let know 5
zapier com 5
https zapier com 4
https zapier 4
jira issue 4
com app editor 3
peter adams 3
peter adams peter 3
peter pellcomp 3

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Group Name: 9

zapier com 15
com app 8
https zapier 8
https zapier com 8
zapier com app 8
com app editor 6
app editor 6
zap https 3
zap https zapier 3
contact zapier 3

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Group Name: 2

com mobile 650 6
passbase email mathias 6
passbase email 6
founder passbase email 6
klenk founder passbase 6
passbase com mobile 6
passbase com 6
650 440 6
mobile 650 440 6
website blog twitter 6

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Group Name: 4

thanks naveen koka 3
naveen koka 3
thanks naveen 3
update issue 1
megapolis sohna 0
megapolis sohna road 0
messaging platform 0
messaging platform helps 0
mobile 650 0
mobile 650 440 0

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Group Name: 1

chris allen 20
allen harvardbusiness org 15
allen harvardbusiness 15
chris allen harvardbusiness 15
harvardbus

,group_name,top_words_and_counts
0,0,"{'pellcomp uk': 6, 'let know': 5, 'zapier com'..."
0,9,"{'zapier com': 15, 'com app': 8, 'https zapier..."
0,2,"{'com mobile 650': 6, 'passbase email mathias'..."
0,4,"{'thanks naveen koka': 3, 'naveen koka': 3, 't..."
0,1,"{'chris allen': 20, 'allen harvardbusiness org..."
0,5,"{'intended recipient': 6, 'dynamicssquare com'..."
0,3,"{'beth bersson': 6, 'manager furthered': 3, '6..."
0,7,"{'ve cc email': 2, 'week client': 2, 've cc': ..."
0,8,"{'joco comhttp www': 2, 'acquisition consultan..."
0,6,"{'quickbuyproperties za': 6, 'http www quickbu..."


## Experiments

In [49]:
e = dt.Experiments()

In [50]:
e.assign_condition_by_id(123432, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'control'

In [51]:
e.assign_condition_by_id(123432, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'control'

In [52]:
e.assign_condition_by_id(123432, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'control'

Note that the same id always gets assigned to the same condition

In [53]:
e.assign_condition_by_id(123432, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'control'

In [54]:
e.assign_condition_by_id(1424125, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'treatment'

In [55]:
e.assign_condition_by_id(1424125, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'treatment'

In [56]:
e.assign_condition_by_id(1424125, np.array(['treatment', 'control']), np.array([0.5, 0.5]), 24)

'treatment'